# Template Pipeline
Normalisation&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**-**  
Imputation 1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Outlier Detection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Z Score**  
Imputation 2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Median**  
Feature Selection&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Implicit to model**  
Model&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**Lasso Regression**


## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
# General
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ML
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE

import xgboost as xgb

# Custom
import sys,os
# %cd /content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution
# sys.path.append('/content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution')
sys.path.append('/home/jovyan/work/AML/Task_1/Solution') # I would like a cleaner solution but works for now
# import Components.Imputation as Imputation
import Components.Outlier_Detection_1D as Outlier_Detection_1D
# import Components.Outlier_Detection_ND as Outlier_Detection_ND
# import Components.Feature_Selection as Feature_Selection
# import Components.Normalisation as Normalisation
import Components.data_fetching as data_fetching

# # CAREFUL:
# # If you make changes to a custom module, you have to reload it, i.e rerun this cell
import importlib
# importlib.reload(Imputation)
importlib.reload(Outlier_Detection_1D)
# importlib.reload(Outlier_Detection_ND)
# importlib.reload(Feature_Selection)
# importlib.reload(Normalisation)
importlib.reload(data_fetching)

<module 'Components.data_fetching' from '/home/jovyan/work/AML/Task_1/Solution/Components/data_fetching.py'>

## Pipeline Optimisatoin

In [ ]:
X,y = data_fetching.get_train_data()
y = np.ravel(y)
x_test_final = data_fetching.get_test_data()

In [ ]:
nrm = QuantileTransformer(output_distribution='normal')
out = Outlier_Detection_1D.Z_Score_Outlier()
imp = IterativeImputer(missing_values=np.nan, max_iter=10, initial_strategy='median' ,random_state=0)
# imp = SimpleImputer(strategy='mean')
rfe = RFE(Ridge())
boost = xgb.XGBRegressor(objective='reg:squarederror')

pipe = Pipeline(steps=[('normalise', nrm),
                        ('z_score', out),
                        ('impute', imp),
                        ('feature_select', rfe),
                        ('regress', boost)])

# Normalisation
n_quants = [50,75]
# Outlier Detection 1d
threashold = [2.75,3]
# Imputer
n_nearest_features = [10,20]
# Feature Selection
n_features = [50,60,70]
# Model
max_depth = [5,6,7]
learning_rate = [0.05, 0.15]
n_estimators = [100, 200]

parameters = dict(normalise__n_quantiles=n_quants,
                    z_score__threashold=threashold,
                    impute__n_nearest_features=n_nearest_features,
                    feature_select__n_features_to_select=n_features,
                    regress__max_depth=max_depth,
                    regress__learning_rate=learning_rate,
                    regress__n_estimators=n_estimators)

clf = GridSearchCV(pipe, parameters)
clf.fit(X, y)

# View The Best Parameters
print('n_quantiles=', clf.best_estimator_.get_params()['normalise__n_quantiles'])
print('threashold=', clf.best_estima/tor_.get_params()['z_score__n_nearest_features'])
print('n_nearest_features=', clf.best_estima/tor_.get_params()['impute__n_nearest_features'])
print('n_features_to_select=', clf.best_estimator_.get_params()['feature_select__n_features_to_select'])
print('max_depth=', clf.best_estimator_.get_params()['regress__max_depth'])
print('learning_rate=', clf.best_estimator_.get_params()['regress__learning_rate'])
print('n_estimators=', clf.best_estimator_.get_params()['regress__n_estimators'])

# Output
# n_quantiles= 50
# n_features_to_select= 50
# learning_rate= 0.05
# n_estimators= 200

## Final Prediction

In [ ]:
y_pred = clf.predict(x_test_final)
plt.hist(y_pred)
print("Train Score:", r2_score(y, clf.predict(X)))

y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
y_pred_pd.to_csv('../Predictions/XGBoost_Optimised.csv', index_label='id')